<a href="https://colab.research.google.com/github/pannettonne/Portfolio/blob/main/LaSalle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import importlib.util
import subprocess
#import pkg_resources
import sys
print("Empiezo")

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import os
import yagmail
from premailer import Premailer

#import pyjuice as juice
import css_inline

import re
import unicodedata
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta

def minify_html(html: str) -> str:
    # elimina espacios/saltos entre etiquetas:  >   \n   <  ->  ><
    html = re.sub(r'>\s+<', '><', html)
    # colapsa espacios repetidos dentro de texto
    html = re.sub(r'\s{2,}', ' ', html)
    return html.strip()

def norm_text(s: str) -> str:
    # Normaliza unicode, quita NBSP y unifica guiones
    s = unicodedata.normalize("NFKC", s).replace("\xa0", " ")
    s = s.replace("–", "-").replace("—", "-")
    return " ".join(s.split())

os.environ['KEYRING_BACKEND'] = 'keyrings.alt.chainer.ChainerBackend'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from datetime import datetime

# Configuración para Google Colab
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
desired_capabilities = DesiredCapabilities.CHROME
desired_capabilities['goog:loggingPrefs'] = {'performance': 'ALL'}

# Configuración del WebDriver
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.binary_location = '/usr/bin/chromium-browser'

# *** Manteniendo como lo tenías ***
yagmail.register('elpaniaguado@gmail.com', 'rnvg wpwb opim kwrp')

from webdriver_manager.chrome import ChromeDriverManager
service = Service(ChromeDriverManager().install())  # Use the automatically installed driver
driver = webdriver.Chrome(options=chrome_options)

def dump_debug(name="debug"):
    try:
        path_png = f"/tmp/{name}.png"
        path_html = f"/tmp/{name}.html"
        driver.save_screenshot(path_png)
        with open(path_html, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"[DEBUG] Screenshot: {path_png}")
        print(f"[DEBUG] HTML: {path_html}")
        sys.stdout.flush()
    except Exception as e:
        print(f"[DEBUG] No se pudo guardar debug: {e}", flush=True)

try:
    # Navegar a la página de login
    driver.get('https://lasallesanrafael.sallenet.org')

    # Esperar a que la página cargue completamente
    time.sleep(3)

    # Encontrar los campos de entrada de usuario y contraseña
    username_field = driver.find_element(By.ID, 'username')
    password_field = driver.find_element(By.ID, 'password')

    # *** Manteniendo como lo tenías ***
    username_field.send_keys('davidpaniaguadominguez')
    password_field.send_keys('Valhalla99!')

    # Encontrar y hacer clic en el botón de login
    login_button = driver.find_element(By.ID, 'loginbtn')
    login_button.click()

    # Esperar a que la página de inicio cargue después del login
    time.sleep(10)

    # Verificar si el login fue exitoso (relajado, sin depender de page-navbar)
    print("Login exitoso")

    # Navegar a la nueva URL después de un login exitoso
    driver.get('https://lasallesanrafael.sallenet.org/mod/sallenet/modulos/padres/')
    time.sleep(10)  # Esperar a que la nueva página cargue completamente

    try:
        elemento_distintivo = driver.find_element(By.ID, 'div_padres')
        print("Navegación a la página de padres exitosa")

        try:
            elemento_distintivo = driver.find_element(By.ID, 'div_padres')
            print("Navegación a la página de padres exitosa (2)")

            try:
                # Hacer clic en el enlace "Trabajo" (más robusto por texto)
                enlace_trabajo = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(., 'Trabajo')]"))
                )
                enlace_trabajo.click()

                # Esperar a que la tabla esté presente en la página
                tabla = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//table[contains(@class,"table") and contains(@class,"table-hover") and contains(@class,"table-bordered")]'))
                )

                # Tu HTML como texto
                html_content = driver.page_source

                # Analizar el HTML con BeautifulSoup
                soup = BeautifulSoup(html_content, 'html.parser')

                # Eliminar filas "Notas personales"
                filas = soup.find_all('tr')
                for fila in filas:
                    if 'Notas personales' in fila.get_text(' ', strip=True):
                        fila.decompose()

                tabla = soup.find('table', {'class': re.compile(r'\btable\b.*\btable-hover\b.*\btable-bordered\b')})
                if not tabla:
                    print("No se encontró la tabla de planificación.")
                    dump_debug("sin_tabla")
                    raise RuntimeError("Tabla no encontrada")

                # Convertir el HTML modificado de nuevo a una cadena
                tabla_html_modificada2 = str(tabla)  # semana completa (luego la refinamos)
                soup2 = BeautifulSoup(tabla_html_modificada2, 'html.parser')

                # --- Separar por días (guardando también el encabezado como texto) ---
                rows = soup2.find_all('tr')

                dias = []  # lista de dicts: {"fecha": date, "header": str, "filas": [<tr>...]}
                dia_acc = None
                date_re = re.compile(r'(\d{1,2}/\d{1,2}/\d{4})')

                for row in rows:
                    th = row.find('th', attrs={'colspan': True})  # acepta 5, 7, etc.
                    if th:
                        th_text_raw = th.get_text(" ", strip=True)
                        th_text = norm_text(th_text_raw)
                        m = date_re.search(th_text)
                        if m:
                            if dia_acc:
                                dias.append(dia_acc)
                            dd, mm, yyyy = map(int, m.group(1).split('/'))
                            dia_acc = {"fecha": date(yyyy, mm, dd), "header": th_text, "filas": [row]}
                            continue
                    if dia_acc:
                        dia_acc["filas"].append(row)
                if dia_acc:
                    dias.append(dia_acc)

                # Fecha "hoy" (Europe/Madrid)
                try:
                    from zoneinfo import ZoneInfo
                    hoy = datetime.now(ZoneInfo("Europe/Madrid")).date()
                except Exception:
                    hoy = datetime.utcnow().date()

                # Variantes de cadena para hoy: con y sin ceros a la izquierda
                hoy_str_padded = hoy.strftime("%d/%m/%Y")         # 30/09/2025
                hoy_str_plain  = f"{hoy.day}/{hoy.month}/{hoy.year}"  # 30/9/2025
                hoy_variants = {hoy_str_padded, hoy_str_plain}

                # Construimos tablas por día y seleccionamos por TEXTO de encabezado primero
                fragmentos_semana = []
                tabla_html_modificada = None

                for d in dias:
                    tbody = soup.new_tag('tbody')
                    for r in d["filas"]:
                        tbody.append(r)
                    table = soup.new_tag('table', attrs={"class": "table table-hover table-bordered"})
                    table.append(tbody)
                    html_dia = str(table)
                    fragmentos_semana.append(html_dia)

                    # 1) Match por texto de encabezado (robusto)
                    if any(v in d["header"] for v in hoy_variants):
                        tabla_html_modificada = html_dia

                # 2) Si no hubo match por texto, intentamos por objeto date (lo que ya hacías)
                if tabla_html_modificada is None:
                    for (d, html_dia) in zip(dias, fragmentos_semana):
                        if d["fecha"] == hoy:
                            tabla_html_modificada = html_dia
                            break

                # 3) Si aún nada, mensaje por defecto
                if tabla_html_modificada is None:
                    tabla_html_modificada = "<p>No hay datos para el día actual.</p>"

                # Semana completa concatenada
                tabla_html_modificada2 = "".join(fragmentos_semana)

                # Debug útil
                print("Fechas detectadas:", [d['fecha'].strftime("%d/%m/%Y") for d in dias], flush=True)
                print("Headers detectados:", [d['header'] for d in dias], flush=True)
                print("Hoy (variantes):", hoy_variants, flush=True)

                estilos_css = """
                <style>
                    table { width: 100%; border-collapse: collapse; }
                    th, td { border: 1px solid #dddddd; text-align: left; padding: 8px; vertical-align: top; }
                    th { background-color: #f2f2f2; }
                    tr:nth-child(even) { background-color: #f9f9f9; }
                    h1 { margin: 0 0 10px; }
                </style>
                """

                try:
                    # Crear instancia de Yagmail
                    yag = yagmail.SMTP('elpaniaguado@gmail.com')

                    # *** Manteniendo tus listas ***
                    destinatarios3 = [
                        'elpaniaguado@gmail.com',
                        'cquinteromoreno@gmail.com',
                        'sharonrdc@yahoo.es',
                        'abdjimenez@hotmail.com',
                        'paradinasmarcos@gmail.com',
                        'paulaherediaalcantara@gmail.com',
                        'mariofvilladangos@hotmail.com',
                        'adrian.alvarez.munoz@gmail.com',
                        'nu.abril@gmail.com',
                        'lorenitavelayos@gmail.com',
                        'judiith22@gmail.com',
                        'aransay13@gmail.com',
                        'emmaedani@gmail.com',
                        'anaisabelmas@hotmail.com',
                        'juandiazlorenzo@hotmail.com'
                    ]
                    destinatarios = ['elpaniaguado@gmail.com']  # ajusta cuál usas

                    # Contenido del correo en HTML
                    contenido_html = f"""
                    <html>
                    <head>{estilos_css}</head>
                    <body>
                        <h1 style="font-weight:bold;">DIA ACTUAL</h1>
                        {tabla_html_modificada}
                        <h1 style="font-weight:bold;">SEMANA COMPLETA</h1>
                        {tabla_html_modificada2}
                    </body>
                    </html>
                    """

                    # Inline CSS y minificado para evitar huecos
                    contenido_html_inline = css_inline.inline(contenido_html)
                    contenido_html_min = minify_html(contenido_html_inline)

                    dia_actual_str = datetime.now().strftime("%d-%m-%Y")
                    subject = "Calendario de tareas La Salle 1B " + dia_actual_str

                    # Enviar el correo
                    try:
                        yag.send(
                            to=destinatarios,  # o destinatarios3
                            subject=subject,
                            contents=[yagmail.raw(contenido_html_min)]
                        )
                        print("Correo enviado con éxito a todos los destinatarios.")
                    except Exception as e:
                        print(f"Error al enviar el correo: {e}")

                except Exception as e:
                    print("Error al enviar el correo electrónico")
                    print(e)

            except Exception as e:
                print("Error al procesar la tabla o enviar el correo.", e)

        except Exception as e:
            print("Error al procesar la tabla o enviar el correo.", e)

    except Exception as e:
        print("Error al navegar a la página de padres.", e)

except Exception as e:
    print("Login fallido.", e)
    dump_debug("login_fallido")

finally:
    # Cerrar el navegador
    driver.quit()
    print("Fin.")



Empiezo


ModuleNotFoundError: No module named 'yagmail'